In [1]:
print("Hello World :)")

Hello World :)


In [2]:
get_ipython().system('pip install db-dtypes')

In [3]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# Constants
PRED_WEEK = 40

In [5]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [6]:
query_get_tables = """
SELECT * 
FROM `ironhacks-data.ironhacks_competition.INFORMATION_SCHEMA.TABLES`
"""
query_job = bigquery_client.query(query_get_tables)
query_job.to_dataframe().head()

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,snapshot_time_ms,ddl,default_collation_name,upsert_stream_apply_watermark
0,ironhacks-data,ironhacks_competition,wage_data,BASE TABLE,YES,NO,2022-11-29 17:00:10.578000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
1,ironhacks-data,ironhacks_competition,prediction_list,BASE TABLE,YES,NO,2022-11-29 17:00:10.722000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
2,ironhacks-data,ironhacks_competition,unemployment_data,BASE TABLE,YES,NO,2022-11-29 17:00:14.588000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT


In [7]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [8]:
query_job = bigquery_client.query(query)
unemployment_data = query_job.to_dataframe()
unemployment_data.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,...,<NA>,<NA>,0,0,0,11,0,0,<NA>,<NA>
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,...,76,35,0,0,0,<NA>,<NA>,0,0,0


In [9]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [10]:
query_job = bigquery_client.query(query)
prediction_list_data = query_job.to_dataframe()
prediction_list_data.head(2)

,uu_id,week_number
0,5bf51fc2e162d6faf9e3cf79e4198378,44
1,420b44cc7e3f55d738df565421e59941,44


In [11]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [12]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()
wage_data.head(2)

,uu_id,countyfips,tract,tract_name,average_wage
0,585f8731c2255d6b3f817a31180848b9,18177,200,"Census Tract 2, Wayne County, Indiana",6612.00
1,8c9d2aa90948679972a9382aadcc6001,18177,900,"Census Tract 9, Wayne County, Indiana",9883.25


In [13]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
WHERE uu_id IN (SELECT uu_id
                FROM `ironhacks-data.ironhacks_competition.prediction_list`
                WHERE week_number = (SELECT MAX(week_number) FROM `ironhacks-data.ironhacks_competition.prediction_list`))
ORDER BY uu_id,week_number
"""

In [14]:
query_job = bigquery_client.query(query)
relevant_unemployment_df = query_job.to_dataframe()
relevant_unemployment_df.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [15]:
relevant_unemployment_df.head(3)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10
2,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [16]:
# Drop duplicate columns
relevant_unemployment_df.drop_duplicates(inplace=True)
relevant_unemployment_df.drop(['timeperiod', 'tract_name'], axis=1, inplace=True)

In [17]:
# Drop columns with excessive null values
# NOTE: Revisit this, these columns may still be useful, especially those that aren't missing too many values
relevant_unemployment_df.dropna(axis='columns', inplace=True)
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()
relevant_unemployment_df['countyfips'] = relevant_unemployment_df['countyfips'].astype(str)
relevant_unemployment_df['tract'] = relevant_unemployment_df['tract'].astype(str)
relevant_unemployment_df['real_data'] = True
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()

Add rows such that every uu_id has rows for weeks 1:max(week_number)

In [18]:
# Add Wage data
relevant_unemployment_df['wage_data'] = relevant_unemployment_df['uu_id'].map(wage_data.set_index('uu_id')['average_wage'])
relevant_unemployment_df.info()
bad_wage_uu_ids = list(set(relevant_unemployment_df.loc[relevant_unemployment_df['wage_data'].isna(),'uu_id']))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13977 entries, 0 to 16831
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   13977 non-null  string 
 1   week_number             13977 non-null  Int64  
 2   countyfips              13977 non-null  string 
 3   tract                   13977 non-null  string 
 4   total_claims            13977 non-null  Int64  
 5   top_category_employer1  13977 non-null  string 
 6   top_category_employer2  13977 non-null  string 
 7   top_category_employer3  13977 non-null  string 
 8   real_data               13977 non-null  boolean
 9   wage_data               13977 non-null  float64
dtypes: Int64(2), boolean(1), float64(1), string(6)
memory usage: 1.1 MB


In [19]:
for uuid in set(relevant_unemployment_df["uu_id"]):
    num_tracts = 0
    tracts=[]
    for tract in set(relevant_unemployment_df.loc[relevant_unemployment_df["uu_id"]==uuid,'wage_data']):
        num_tracts += 1
        tracts.append(tract)
    if num_tracts > 1:
        print(uuid,num_tracts,tracts)
relevant_unemployment_df

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,001cd9ae23064d7f0fd3cd327c873d8d,31,18099,20102,34,31-33,23,81,True,10484.333333
1,001cd9ae23064d7f0fd3cd327c873d8d,33,18099,20102,10,31-33,23,81,True,10484.333333
3,001cd9ae23064d7f0fd3cd327c873d8d,34,18099,20102,25,62,31-33,23,True,10484.333333
5,001cd9ae23064d7f0fd3cd327c873d8d,35,18099,20102,10,31-33,61,23,True,10484.333333
7,001cd9ae23064d7f0fd3cd327c873d8d,36,18099,20102,10,31-33,23,81,True,10484.333333
...,...,...,...,...,...,...,...,...,...,...
16823,fec479d0202d6e1e3f051a9ee902ff5d,33,18089,43300,59,62,23,31-33,True,9908.312500
16825,fec479d0202d6e1e3f051a9ee902ff5d,34,18089,43300,34,23,62,61,True,9908.312500
16827,fec479d0202d6e1e3f051a9ee902ff5d,35,18089,43300,27,62,23,72,True,9908.312500
16829,fec479d0202d6e1e3f051a9ee902ff5d,36,18089,43300,28,23,62,72,True,9908.312500


In [20]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [21]:
temp_df = pd.DataFrame(columns=list(relevant_unemployment_df.columns))
my_ids = list(set(relevant_unemployment_df['uu_id']))
for UUid in my_ids:
    ctyfips = None
    wage_data = None
    tract_vote = [None, 0]
    top_cat_1_vote = [None, 0]
    top_cat_2_vote = [None, 0]
    top_cat_3_vote = [None, 0]
    total_claims_avg = []
    total_claims_num = 0
    
    for week in range(1,PRED_WEEK):
        row = relevant_unemployment_df.loc[((relevant_unemployment_df.uu_id==UUid) 
                                      & (relevant_unemployment_df.week_number == week)),]
        if len(row) == 1:
            temp_df = pd.concat([temp_df, row])
            
            ### Store information about values to infer for rows without data
            ## Consistent within uu_id values
            ctyfips = row['countyfips'].values[0]
            wage_data = row['wage_data'].values[0]
            
            ## Inconsistent within uu_id values
            # Categorical 

            if ((tract_vote[0] == None) or ((tract_vote[0] != row['tract'].values[0]) and (tract_vote[1] == 1))):
                tract_vote = [row['tract'].values[0], 1]
            elif tract_vote[0] == row['tract'].values[0]:
                tract_vote[1] += 1
            else:
                tract_vote[1] -= 1
                
            if ((top_cat_1_vote[0] == None) or ((top_cat_1_vote[0] != row['top_category_employer1'].values[0]) and (top_cat_1_vote[1] == 1))):
                top_cat_1_vote = [row['top_category_employer1'].values[0], 1]
            elif top_cat_1_vote[0] == row['top_category_employer1'].values[0]:
                top_cat_1_vote[1] += 1
            else:
                top_cat_1_vote[1] -= 1
                
            if ((top_cat_2_vote[0] == None) or ((top_cat_2_vote[0] != row['top_category_employer2'].values[0]) and (top_cat_2_vote[1] == 1))):
                top_cat_2_vote = [row['top_category_employer2'].values[0], 1]
            elif top_cat_2_vote[0] == row['top_category_employer2'].values[0]:
                top_cat_2_vote[1] += 1
            else:
                top_cat_2_vote[1] -= 1
                
            if ((top_cat_3_vote[0] == None) or ((top_cat_3_vote[0] != row['top_category_employer3'].values[0]) and (top_cat_3_vote[1] == 1))):
                top_cat_3_vote = [row['top_category_employer3'].values[0], 1]
            elif top_cat_3_vote[0] == row['top_category_employer3'].values[0]:
                top_cat_3_vote[1] += 1
            else:
                top_cat_3_vote[1] -= 1
                
            # Numerical
            total_claims_avg.append(row['total_claims'].values[0])
            if len(total_claims_avg) > 0:
                total_claims_num = np.mean(total_claims_avg)

            continue
        temp_df = pd.concat([temp_df, pd.DataFrame.from_records([{'uu_id': UUid, 
                                                                  'week_number': week, 
                                                                  'countyfips': ctyfips, 
                                                                  'tract': tract_vote[0],
                                                                  'total_claims': total_claims_num, 
                                                                  'top_category_employer1': top_cat_1_vote[0],
                                                                  'top_category_employer2': top_cat_2_vote[0], 
                                                                  'top_category_employer3': top_cat_3_vote[0],
                                                                  'real_data': False, 
                                                                  'wage_data': wage_data}])])

In [22]:
temp_df.reset_index(inplace=True)
temp_df.drop('index', axis=1,inplace=True)
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20263 non-null  object 
 3   tract                   20263 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20263 non-null  object 
 6   top_category_employer2  20263 non-null  object 
 7   top_category_employer3  20263 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20263 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [23]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,7b208b710114a393f20ec464c7849157,1,18089,12400,41.0,31-33,56,44-45,True,8687.75
1,7b208b710114a393f20ec464c7849157,2,18089,12400,40.0,56,31-33,61,True,8687.75
2,7b208b710114a393f20ec464c7849157,3,18089,12400,27.0,31-33,62,56,True,8687.75
3,7b208b710114a393f20ec464c7849157,4,18089,12400,36.0,31-33,62,56,False,8687.75
4,7b208b710114a393f20ec464c7849157,5,18089,12400,35.0,56,62,53,True,8687.75


In [24]:
non_null_cols = ["countyfips", "tract", "top_category_employer1", "top_category_employer2", "top_category_employer3", "wage_data"]

In [25]:
for UUid2 in my_ids[::-1]:
    for week_num in range(PRED_WEEK-1,0,-1):
        row = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)),]
        if row['countyfips'].values[0] == None:
            for col in non_null_cols:
                temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)), col] = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num+1)), col].values[0]

In [26]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  object 
 3   tract                   20475 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  object 
 6   top_category_employer2  20475 non-null  object 
 7   top_category_employer3  20475 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20475 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [27]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,7b208b710114a393f20ec464c7849157,1,18089,12400,41.0,31-33,56,44-45,True,8687.75
1,7b208b710114a393f20ec464c7849157,2,18089,12400,40.0,56,31-33,61,True,8687.75
2,7b208b710114a393f20ec464c7849157,3,18089,12400,27.0,31-33,62,56,True,8687.75
3,7b208b710114a393f20ec464c7849157,4,18089,12400,36.0,31-33,62,56,False,8687.75
4,7b208b710114a393f20ec464c7849157,5,18089,12400,35.0,56,62,53,True,8687.75


In [28]:
temp_df = temp_df.convert_dtypes()
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  string 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  string 
 3   tract                   20475 non-null  string 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  string 
 6   top_category_employer2  20475 non-null  string 
 7   top_category_employer3  20475 non-null  string 
 8   real_data               20475 non-null  boolean
 9   wage_data               20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), string(6)
memory usage: 1.5 MB


In [29]:
bad_wage_ids=set(temp_df.loc[temp_df['wage_data'].isna(),"uu_id"])

In [30]:
data = temp_df
data.rename(columns={"uu_id": "id", "week_number": "week", "countyfips": "fips", "top_category_employer1": "emp1"}, inplace=True)
data.rename(columns={"top_category_employer2": "emp2", "top_category_employer3": "emp3", "real_data": "real", "wage_data":"wages"}, inplace=True)
for col in ['fips', 'tract', 'emp1', 'emp2', 'emp3']:
    data[col] = pd.factorize(data[col])[0]

In [31]:
data.head()

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,7b208b710114a393f20ec464c7849157,1,0,0,41.0,0,0,0,True,8687.75
1,7b208b710114a393f20ec464c7849157,2,0,0,40.0,1,1,1,True,8687.75
2,7b208b710114a393f20ec464c7849157,3,0,0,27.0,0,2,2,True,8687.75
3,7b208b710114a393f20ec464c7849157,4,0,0,36.0,0,2,2,False,8687.75
4,7b208b710114a393f20ec464c7849157,5,0,0,35.0,1,2,3,True,8687.75


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20475 non-null  string 
 1   week          20475 non-null  Int64  
 2   fips          20475 non-null  int64  
 3   tract         20475 non-null  int64  
 4   total_claims  20475 non-null  Float64
 5   emp1          20475 non-null  int64  
 6   emp2          20475 non-null  int64  
 7   emp3          20475 non-null  int64  
 8   real          20475 non-null  boolean
 9   wages         20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), int64(5), string(1)
memory usage: 1.5 MB


In [33]:
abs(data.corr()['total_claims'])

week            0.042895
fips            0.178031
tract           0.002011
total_claims    1.000000
emp1            0.003813
emp2            0.030294
emp3            0.044094
real            0.165613
wages           0.123782
Name: total_claims, dtype: float64

In [34]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]

In [35]:
from sklearn.linear_model import LinearRegression

In [36]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y)
        
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction)

In [37]:
predictions

{'7b208b710114a393f20ec464c7849157': [array([55.8627655]),
  array([46.71639544]),
  array([39.56017115]),
  array([26.24471496]),
  array([46.41853609]),
  array([40.34292155]),
  array([24.16649078]),
  array([37.60272276]),
  array([37.3862517])]}

In [38]:
from sklearn.linear_model import LinearRegression

In [39]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y)
        
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

In [40]:
predictions

{'7b208b710114a393f20ec464c7849157': [55.86276549611323,
  46.71639544454787,
  39.56017114578153,
  26.244714957989753,
  46.41853608576198,
  40.342921552263675,
  24.166490780409983,
  37.60272276393938,
  37.38625169710644]}

In [41]:
from sklearn.linear_model import LinearRegression

In [42]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y)
        print(train_x)
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

    fips  tract  emp1  emp2  emp3    wages
0      0      0     0     0     0  8687.75
1      0      0     1     1     1  8687.75
2      0      0     0     2     2  8687.75
3      0      0     0     2     2  8687.75
4      0      0     1     2     3  8687.75
5      0      0     1     3     4  8687.75
6      0      0     0     0     5  8687.75
7      0      0     0     0     6  8687.75
8      0      0     1     1     1  8687.75
9      0      0     2     0     0  8687.75
10     0      0     3     0     4  8687.75
11     0      0     1     4     6  8687.75
12     0      0     1     3     0  8687.75
13     0      0     1     1     6  8687.75
14     0      0     1     4     7  8687.75
15     0      0     1     5     1  8687.75
16     0      0     1     4     4  8687.75
17     0      0     4     6     7  8687.75
18     0      0     5     0     5  8687.75
19     0      0     0     4     2  8687.75
20     0      0     0     4     1  8687.75
21     0      0     6     4     8  8687.75
22     0   

In [43]:
from sklearn.linear_model import LinearRegression

In [44]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y)
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

In [45]:
actual

{'7b208b710114a393f20ec464c7849157': [30    32.0
  Name: total_claims, dtype: Float64,
  31    21.0
  Name: total_claims, dtype: Float64,
  32    21.0
  Name: total_claims, dtype: Float64,
  33    17.0
  Name: total_claims, dtype: Float64,
  34    11.0
  Name: total_claims, dtype: Float64,
  35    18.0
  Name: total_claims, dtype: Float64,
  36    25.0
  Name: total_claims, dtype: Float64,
  37    34.628571
  Name: total_claims, dtype: Float64,
  38    34.628571
  Name: total_claims, dtype: Float64]}

In [46]:
from sklearn.linear_model import LinearRegression

In [47]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y['total_claims']).values[0]
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

KeyError: 'total_claims'

In [48]:
from sklearn.linear_model import LinearRegression

In [49]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y['total_claims'].values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

KeyError: 'total_claims'

In [50]:
from sklearn.linear_model import LinearRegression

In [51]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y)
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

In [52]:
predictions

{'7b208b710114a393f20ec464c7849157': [55.86276549611323,
  46.71639544454787,
  39.56017114578153,
  26.244714957989753,
  46.41853608576198,
  40.342921552263675,
  24.166490780409983,
  37.60272276393938,
  37.38625169710644]}

In [53]:
actual

{'7b208b710114a393f20ec464c7849157': [30    32.0
  Name: total_claims, dtype: Float64,
  31    21.0
  Name: total_claims, dtype: Float64,
  32    21.0
  Name: total_claims, dtype: Float64,
  33    17.0
  Name: total_claims, dtype: Float64,
  34    11.0
  Name: total_claims, dtype: Float64,
  35    18.0
  Name: total_claims, dtype: Float64,
  36    25.0
  Name: total_claims, dtype: Float64,
  37    34.628571
  Name: total_claims, dtype: Float64,
  38    34.628571
  Name: total_claims, dtype: Float64]}

In [54]:
from sklearn.linear_model import LinearRegression

In [55]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

KeyError: 0

In [56]:
predictions

{'7b208b710114a393f20ec464c7849157': []}

In [57]:
from sklearn.linear_model import LinearRegression

In [58]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

In [59]:
predictions

{'7b208b710114a393f20ec464c7849157': [55.86276549611323,
  46.71639544454787,
  39.56017114578153,
  26.244714957989753,
  46.41853608576198,
  40.342921552263675,
  24.166490780409983,
  37.60272276393938,
  37.38625169710644]}

In [60]:
actual

{'7b208b710114a393f20ec464c7849157': [32.0,
  21.0,
  21.0,
  17.0,
  11.0,
  18.0,
  25.0,
  34.628571428571426,
  34.628571428571426]}

In [61]:
from sklearn.linear_model import LinearRegression

In [62]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            print(train_x
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

SyntaxError: invalid syntax (4234595879.py, line 18)

In [63]:
from sklearn.linear_model import LinearRegression

In [64]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            print(train_x)
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

In [65]:
from sklearn.linear_model import LinearRegression

In [66]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        print(train_x)
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

    fips  tract  emp1  emp2  emp3    wages
0      0      0     0     0     0  8687.75
1      0      0     1     1     1  8687.75
2      0      0     0     2     2  8687.75
3      0      0     0     2     2  8687.75
4      0      0     1     2     3  8687.75
5      0      0     1     3     4  8687.75
6      0      0     0     0     5  8687.75
7      0      0     0     0     6  8687.75
8      0      0     1     1     1  8687.75
9      0      0     2     0     0  8687.75
10     0      0     3     0     4  8687.75
11     0      0     1     4     6  8687.75
12     0      0     1     3     0  8687.75
13     0      0     1     1     6  8687.75
14     0      0     1     4     7  8687.75
15     0      0     1     5     1  8687.75
16     0      0     1     4     4  8687.75
17     0      0     4     6     7  8687.75
18     0      0     5     0     5  8687.75
19     0      0     0     4     2  8687.75
20     0      0     0     4     1  8687.75
21     0      0     6     4     8  8687.75
22     0   

In [67]:
from sklearn.linear_model import LinearRegression

In [68]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        print(train_y)
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

0          41.0
1          40.0
2          27.0
3          36.0
4          35.0
5          28.0
6          36.0
7          42.0
8          29.0
9          30.0
10        102.0
11         37.0
12         13.0
13         62.0
14         75.0
15         23.0
16         23.0
17        140.0
18         33.0
19         30.0
20         23.0
21         19.0
22    42.285714
23         25.0
24         35.0
25         49.0
26         12.0
27         15.0
28         21.0
29         22.0
30         32.0
31         21.0
32         21.0
33         17.0
34         11.0
35         18.0
36         25.0
37    34.628571
Name: total_claims, dtype: Float64
0          41.0
1          40.0
2          27.0
3          36.0
4          35.0
5          28.0
6          36.0
7          42.0
8          29.0
9          30.0
10        102.0
11         37.0
12         13.0
13         62.0
14         75.0
15         23.0
16         23.0
17        140.0
18         33.0
19         30.0
20         23.0
21         19.0
22   

In [69]:
from sklearn.linear_model import LinearRegression

In [70]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        print(pred_id)
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157


In [71]:
temp_df

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,7b208b710114a393f20ec464c7849157,1,0,0,41.0,0,0,0,True,8687.75
1,7b208b710114a393f20ec464c7849157,2,0,0,40.0,1,1,1,True,8687.75
2,7b208b710114a393f20ec464c7849157,3,0,0,27.0,0,2,2,True,8687.75
3,7b208b710114a393f20ec464c7849157,4,0,0,36.0,0,2,2,False,8687.75
4,7b208b710114a393f20ec464c7849157,5,0,0,35.0,1,2,3,True,8687.75
...,...,...,...,...,...,...,...,...,...,...
20470,f45fb7ce83b62c22c577bca121fdb2c1,35,22,424,13.0,0,0,6,True,11397.666667
20471,f45fb7ce83b62c22c577bca121fdb2c1,36,22,424,48.0,5,7,2,True,11397.666667
20472,f45fb7ce83b62c22c577bca121fdb2c1,37,22,424,14.0,1,8,9,True,11397.666667
20473,f45fb7ce83b62c22c577bca121fdb2c1,38,22,424,21.121212,9,8,9,False,11397.666667


In [72]:
temp_df.head(39)

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,7b208b710114a393f20ec464c7849157,1,0,0,41.0,0,0,0,True,8687.75
1,7b208b710114a393f20ec464c7849157,2,0,0,40.0,1,1,1,True,8687.75
2,7b208b710114a393f20ec464c7849157,3,0,0,27.0,0,2,2,True,8687.75
3,7b208b710114a393f20ec464c7849157,4,0,0,36.0,0,2,2,False,8687.75
4,7b208b710114a393f20ec464c7849157,5,0,0,35.0,1,2,3,True,8687.75
5,7b208b710114a393f20ec464c7849157,6,0,0,28.0,1,3,4,True,8687.75
6,7b208b710114a393f20ec464c7849157,7,0,0,36.0,0,0,5,True,8687.75
7,7b208b710114a393f20ec464c7849157,8,0,0,42.0,0,0,6,True,8687.75
8,7b208b710114a393f20ec464c7849157,9,0,0,29.0,1,1,1,True,8687.75
9,7b208b710114a393f20ec464c7849157,10,0,0,30.0,2,0,0,True,8687.75


In [73]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [74]:
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
    
    # Error metrics
    MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [75]:
predictions

{'7b208b710114a393f20ec464c7849157': [55.86276549611323,
  46.71639544454787,
  39.56017114578153,
  26.244714957989753,
  46.41853608576198,
  40.342921552263675,
  24.166490780409983,
  37.60272276393938,
  37.38625169710644]}

In [76]:
actual

{'7b208b710114a393f20ec464c7849157': [32.0,
  21.0,
  21.0,
  17.0,
  11.0,
  18.0,
  25.0,
  34.628571428571426,
  34.628571428571426]}

In [77]:
MSE

{'7b208b710114a393f20ec464c7849157': [381.28147062170524]}

In [78]:
MAE

{'7b208b710114a393f20ec464c7849157': [15.745649500661223]}

In [79]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [80]:
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
    
    # Error metrics
    MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [81]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [82]:
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        print(pred_id)
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
    
    # Error metrics
    MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7ee27423441d3bced123afcec6335da8
7ee27423441d3bced123afcec6335da8
7ee27423441d3bced123afcec6335da8
7ee2742344

In [83]:
bad_ids

NameError: name 'bad_ids' is not defined

In [84]:
bad_wage_ids

set()

In [85]:
train_x

,fips,tract,emp1,emp2,emp3,wages
20436,22,424,9,2,1,11397.666667
20437,22,424,9,0,8,11397.666667
20438,22,424,9,2,10,11397.666667
20439,22,424,9,2,10,11397.666667
20440,22,424,9,7,8,11397.666667
20441,22,424,9,19,14,11397.666667
20442,22,424,9,19,2,11397.666667
20443,22,424,9,0,4,11397.666667
20444,22,424,9,2,2,11397.666667
20445,22,424,9,14,2,11397.666667


In [86]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [87]:
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        print(pred_id)
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
    
    # Error metrics
    MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
7b208b710114a393f20ec464c7849157
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
9e943f3dccee65ed33733a794a4d030e
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7df3e38fbdab0a32ca0fa9fb6c8bd8c4
7ee27423441d3bced123afcec6335da8
7ee27423441d3bced123afcec6335da8
7ee27423441d3bced123afcec6335da8
7ee2742344

In [88]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [89]:
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
    
    # Error metrics
    MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [90]:
MSE

{'7b208b710114a393f20ec464c7849157': [381.28147062170524],
 '9e943f3dccee65ed33733a794a4d030e': [113.98131041084633],
 '7df3e38fbdab0a32ca0fa9fb6c8bd8c4': [523.0086630276468],
 '7ee27423441d3bced123afcec6335da8': [53.04222152329629],
 '8eef3f1453830d1939aae4e774e87c53': [109.79784232548721],
 '05f47cc80d60a67d61e891e2b90c2045': [69.4394623922035],
 'b1d41e64e8d10b7c427570a7f77f1826': [121.80229964167708],
 '36c489a05d6a987ff961fdc67ccadbc1': [18.896841324193783],
 '8ed377ac3767ffda176fcdea40bcd7e4': [193.85364313743398],
 '5846c7b267dc6e4b00863277e3582f80': [3.6753577805913458],
 '2a0f380e277560b810399f7c6a01e76e': [243.62757889802455],
 'cdfe175e21e14759b98d9dc9ad03fb10': [132.0463409696564],
 '471301a66d4c4feb1f474cf934be8999': [85.42537198246765],
 'dd240f0580c98aeff68dfc9ab6a19b4a': [6.629577814518706],
 '48ac5846d8d781fd5b7c6a863a572359': [655.5796489571936],
 '6505dba31a2e5fa96251a7ba5d2fc1a0': [13.170005843792275],
 '8d7022c454fc05456cfb4b9cadf5f109': [300.5036192127831],
 'd4ce